In [1]:
import pandas as pd
import numpy as np

In [2]:
df_esail = pd.read_csv('../data/raw/esail.csv')

df_esail.head()

,sett_bmu_id,ngc_bmu_id,bmu_root,name,primary_fuel_type,detailed_fuel_type,longitude,latitude,common_name
0,E_MARK-1,MARK-1,MARK,Rothes Bio-Plant CHP 1,biomass,bone,-3.603516,57.480403,Rothes Bio-Plant CHP
1,E_MARK-2,MARK-2,MARK,Rothes Bio-Plant CHP 2,biomass,bone,-3.603516,57.480403,Rothes Bio-Plant CHP
2,T_DIDC1,DIDC1,DIDC,Didcot A (G) 1,coal,coalgas_opt_out,-1.267570,51.623630,Didcot A (G)
3,T_DIDC2,DIDC2,DIDC,Didcot A (G) 2,coal,coalgas_opt_out,-1.267570,51.623630,Didcot A (G)
4,T_DIDC4,DIDC4,DIDC,Didcot A (G) 4,coal,coalgas_opt_out,-1.267570,51.623630,Didcot A (G)


In [69]:
sett_to_ngc_bmu_id = df_esail.set_index('sett_bmu_id')['ngc_bmu_id'].to_dict()
ngc_to_sett_bmu_id = df_esail.set_index('ngc_bmu_id')['sett_bmu_id'].to_dict()

In [3]:
df_powerdict = pd.read_csv('../data/output/power_stations.csv')

print(df_powerdict.shape)

df_powerdict.head()

(288, 9)


,osuked_id,esail_id,gppd_idnr,name,sett_bmu_id,longitude,latitude,fuel_type,capacity_mw
0,10000,MARK,NaN,Rothes Bio-Plant CHP,"E_MARK-1, E_MARK-2",-3.603516,57.480403,biomass,NaN
1,10001,DIDC,NaN,Didcot A (G),"T_DIDC1, T_DIDC2, T_DIDC4, T_DIDC3",-1.267570,51.623630,coal,NaN
2,10002,ABTH,GBR1000374,Aberthaw B,"T_ABTH7, T_ABTH8, T_ABTH9",-3.404866,51.387312,coal,1586.0
3,10003,COTPS,GBR1000142,Cottam,"T_COTPS-1, T_COTPS-2, T_COTPS-3, T_COTPS-4",-0.648193,53.245495,coal,2008.0
4,10004,DRAXX,GBR0000174,Drax,"T_DRAXX-1, T_DRAXX-2, T_DRAXX-3, T_DRAXX-4, T_...",-0.626221,53.748711,"coal, biomass",1980.0


In [4]:
df_carbon_powerdict = df_powerdict[~df_powerdict['fuel_type'].isin(['wind', 'hydro', 'biomass', 'nuclear'])].sort_values('capacity_mw', ascending=False)
carbon_capacity_share = df_carbon_powerdict['capacity_mw'].sum()/df_powerdict['capacity_mw'].sum()

print(f'CO2 emitting generation makes up {carbon_capacity_share:.0%} of the total capacity in the power-station dictionary')

CO2 emitting generation makes up 67% of the total capacity in the power-station dictionary


In [5]:
df_B1610_sample = pd.read_csv('B1610_sample.csv').set_index('datetime')

df_B1610_sample

,ABRBO-1,ABRTW-1,ABTH7,ABTH7G,ABTH8,ABTH8G,ABTH9,ABTH9G,ACHRW-1,AKGLW-2,...,WILCT-1,WLNYO-2,WLNYO-3,WLNYO-4,WLNYW-1,WTMSO-1,WYLF-1,WYLF-2,WYLF-3,WYLF-4
datetime,,,,,,,,,,,,,,,,,,,,,
2015-10-04 23:00:00,NaN,NaN,421.692,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,132.9,NaN,NaN,95.602,9.858,227.38,212.41,NaN,NaN


In [68]:
flatten_list = lambda t: [item for sublist in t for item in sublist]

powerdict_bmu_ids = flatten_list(df_powerdict['sett_bmu_id'].str.split(', ').to_list())
powerdict_ngc_bmu_ids = pd.Series(powerdict_bmu_ids).map(sett_to_ngc_bmu_id).dropna().to_list()

ids_in_powerdict_but_not_B1610 = list(set(powerdict_ngc_bmu_ids) - set(df_B1610_sample.columns))
ids_in_B1610_but_not_powerdict = list(set(df_B1610_sample.columns) - set(powerdict_ngc_bmu_ids))

print(f'{len(ids_in_B1610_but_not_powerdict)/len(df_B1610_sample.columns):.2%} of the ids in B1610 are not in powerdict')
print(f'{len(ids_in_powerdict_but_not_B1610)/len(powerdict_ngc_bmu_ids):.2%} of the ids in powerdict are not in B1610')

6.17% of the ids in B1610 are not in powerdict
25.18% of the ids in powerdict are not in B1610


In [44]:
sett_bmu_to_osuked_id = {}
sett_bmu_id_to_fuel = {}

for osuked_id, sett_bmu_ids in df_powerdict.set_index('osuked_id')['sett_bmu_id'].items():
    for sett_bmu_id in sett_bmu_ids.split(', '):
        sett_bmu_to_osuked_id[sett_bmu_id] = osuked_id
        sett_bmu_id_to_fuel[sett_bmu_id] = df_powerdict.set_index('osuked_id').loc[osuked_id, 'fuel_type']
        
s_osuked_ids_to_add = pd.Series(ids_in_powerdict_but_not_B1610, index=ids_in_powerdict_but_not_B1610).map(ngc_to_sett_bmu_id).map(sett_bmu_to_osuked_id)

s_osuked_ids_to_add.head()

FAWL3     10140
GRAI2G    10096
GRAI3G    10096
BARK-1    10022
KINO4G    10099
dtype: int64

In [9]:
df_installations = pd.read_csv('https://raw.githubusercontent.com/OSUKED/ETS-Watch/master/data/installations/installations.csv').set_index('account_id')

df_installations.head(3)

,installation_id,installation_name,permit_id,permit_entry_date,permit_expiration_Date,subsidiary_undertaking_name,parent_undertaking_name,EPRTR_id,initial_emissions_year,final_emissions_year,first_address_line,second_address_line,postcode,city,country,lat,lon,main_activity
account_id,,,,,,,,,,,,,,,,,,
93707,47,AGRANA Gmünd,ILE166,2005-01-26,NaN,NaN,NaN,NaN,2005,0,Conrathstrasse 7,NaN,3953,Gmünd,AT,NaN,NaN,20-Combustion of fuels
93708,50,AGRANA Aschach,ILE165,2005-10-18,NaN,NaN,NaN,20000.00442,2005,0,Raiffeisenweg 2-6,NaN,4082,Aschach,AT,NaN,NaN,20-Combustion of fuels
93709,51,AGRANA Tulln,ILE161,2005-07-01,NaN,NaN,NaN,20000.00320,2005,0,Josef-Reither-Strasse 21-23,NaN,3430,Tulln,AT,NaN,NaN,20-Combustion of fuels


In [10]:
df_emissions = pd.read_csv('https://raw.githubusercontent.com/OSUKED/ETS-Watch/master/data/installations/verified_emissions.csv').set_index('account_id')

df_emissions.head(3)

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,...,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
account_id,,,,,,,,,,,,,,,,,,,,,
100000,87923,106745,117404,126296,105795,105034,101784,21384,5671,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100001,134139,138143,170415,143182,149405,142121,119263,102751,101881,99012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,0,NaN,NaN,NaN,Not Reported,Not Reported,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_emissions = (df_emissions
                .replace('Excluded', np.nan)
                .replace('Not Reported', np.nan)
                .astype(float))

assert (df_emissions<0).sum().sum() == 0, 'Negative emissions are present'

df_emissions.head()

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,...,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
account_id,,,,,,,,,,,,,,,,,,,,,
100000,87923.0,106745.0,117404.0,126296.0,105795.0,105034.0,101784.0,21384.0,5671.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100001,134139.0,138143.0,170415.0,143182.0,149405.0,142121.0,119263.0,102751.0,101881.0,99012.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
num_big_emitters = 500

df_gb_combustion_installations = df_installations.query('country=="GB" & (main_activity=="20-Combustion of fuels" | main_activity=="1-Combustion installations with a rated thermal input exceeding 20 MW")')
biggest_emitters = df_emissions.loc[df_gb_combustion_installations.index].mean(axis=1).nlargest(num_big_emitters).index

df_gb_combustion_installations.loc[biggest_emitters].head()

,installation_id,installation_name,permit_id,permit_entry_date,permit_expiration_Date,subsidiary_undertaking_name,parent_undertaking_name,EPRTR_id,initial_emissions_year,final_emissions_year,first_address_line,second_address_line,postcode,city,country,lat,lon,main_activity
account_id,,,,,,,,,,,,,,,,,,
96842,381,Drax Power Station,UK-E-IN-11718,2005-05-18,NaN,NaN,NaN,NaN,2005,0,"DRAX POWER LIMITED,","DRAX POWER STATION, PO BOX 3",YO8 8PQ,"SELBY, NORTH YORKSHIRE",GB,NaN,NaN,20-Combustion of fuels
96923,593,Longannet Power Station,UK-S-IN-12326,2005-05-20,2016-07-01,NaN,NaN,NaN,2005,2016,Longannet Power Station,Spean Street,FK10 4AA,Alloa,GB,NaN,NaN,20-Combustion of fuels
97778,202,Cottam Power Station,UK-E-IN-13569,2005-05-17,2020-03-04,NaN,NaN,NaN,2005,2019,Settlements,"Cardinal Place, 2nd Floor",SW1E 5JL,London,GB,NaN,NaN,20-Combustion of fuels
97052,154,Ratcliffe on Soar power station,UK-E-IN-13306,2005-05-23,NaN,NaN,NaN,NaN,2005,0,Uniper UK Limited,"Westwood Way, Westwood Business Park",CV4 8LG,Coventry,GB,NaN,NaN,20-Combustion of fuels
97220,145,West Burton Power Station,UK-E-IN-11463,2005-05-18,NaN,NaN,NaN,NaN,2005,0,Settlements,"Cardinal Place, 2nd FLoor",SW1E 5JL,London,GB,NaN,NaN,20-Combustion of fuels


In [14]:
# Couldnt find:
# * eutl-98286 (wilton power station)
# * osuked-10080 (bridgewater district energy), some confusion over whether its part of a paper plant, waste to power, or gas as stated in the powerdict
# * osuked-10126 (manx power)

osuked_to_account_id = {
    10004: 96842, # drax (coal/biomass) - looks like biomass doesnt count towards emissions
    10093: 96842, # drax (oil)
    10003: 97778, # cottam
    10009: 96923, # longannet
    10011: 97052, # ratcliffe
    10014: 97220, # west burton a (coal)
    10062: 97223, # west burton b (gas) - not sure if the osuked-10106 oil plant is under this permit too 
    10005: 97445, # eggborough
    10094: 98253, # ferrybridge c (oil)
    10006: 98253, # ferrybridge c (coal)
    10007: 98251, # fiddlers ferry (coal)
    10095: 98251, # fiddlers ferry (oil)
    10002: 97175, # aberthaw b (coal)
    10090: 97175, # aberthaw b (oil)
    10012: 97192, # rugeley (coal)
    10103: 97192, # rugeley (oil)
    10001: 97165, # didcot a (coal)
    10092: 97165, # didcot a (oil)
    10035: 97165, # didcot b (gas) - note that this is a guess and the account only specifies didcot a
    10047: 97190, # pembroke (gas)
    10070: 96880, # immingham (chp)
    10020: 97158, # tilbury b (coal)
    10105: 97158, # tilbury b (oil)
    10056: 96979, # south humber bank (gas)
    10029: 97054, # connahs quay (gas)
    10022: 97034, # barking 1 (gas)
    10023: 97034, # barking 2 (gas)
    10010: 96827, # lynemouth (coal)
    10033: 96906, # damhead creek (gas)
    10067: 97061, # grain (chp)
    10096: 97051, # grain (oil) - guess
    10027: 112433, # carrington (ccgt)
    10018: 97047, # ironbridge (coal)
    10059: 97236, # sutton bridge (gas)
    10039: 98249, # keadby (gas)
    10098: 98249, # keadby (oil)
    10046: 98257, # medway (gas)
    10043: 97585, # langage (gas)
    10054: 97635, # severn power (ccgt)
    10021: 96786, # baglan (gas)
    10068: 97781, # grangemouth (chp)
    10051: 96908, # rye house (gas)
    10032: 97059, # cottam development (gas)
    10003: 97778, # cottam (coal)
    10034: 97636, # deeside (gas)
    10040: 97443, # killingholme (a)
    10041: 97057, # killingholme (b) - guess, if this is incorrect check the capacity for osuked-10040
    10055: 96911, # shoreham (gas)
    10036: 97044, # enfield (gas)
    10042: 116565, # kings lynn (gas)
    10048: 97020, # peterborough (gas)
    10057: 117846, # spalding (gas)
    10024: 97659, # barry (gas)
    10026: 97215, # brigg (gas)
    10104: 97048, # taylors (oil)
    10028: 97683, # castleford (gas)
    10071: 97716, # sandbach (gas)
    10061: 97720, # thornhill (gas)
    10058: 97185, # staythorpe (gas)
    10052: 96903, # saltend (gas)
    10085: 97910, # peterhead (gas)
    10045: 97532, # marchwood (gas)
    10049: 96873, # rocksavage (gas)
    10031: 96934, # coryton (gas)
    10044: 97169, # little barford (gas)
    10038: 97183, # great yarmouth (gas)
    10053: 96899, # seabank (gas)
    10030: 97634, # corby (gas)
    10013: 97413, # uksmouth (coal)
    10097: 96869, # indian queens (oil)
    10091: 97160, # cowes (oil)
    10060: 97075, # teeside (gas)
    10015: 96927, # cockenzie (coal)
    10019: 97055, # (coal)
    10099: 97055, # (oil)
    10037: 97887, # fife (ccgt)
    10050: 96938, # roosecote (gas)
    10064: 97227, # derwent (gas)
    10072: 97674, # shotton (ccgt)
    10074: 97739, # stoke (chp)
    10078: 97150, # aberdare (gas)
    10065: 96962, # fawley cogen (gas)
    10082: 97172, # fawley (gas/oil)
    10140: 97172, # fawley (oil)
    10088: 97145, # sevington (gas)
    10089: 97142, # solutia (gas) - small possibility it could be eutl-96967
    10100: 97177, # littlebrook agt (oil)
    10141: 97177, # littlebrook (oil)
    10085: 97910, # peterhead (gas)
    10101: 97910, # peterhead (oil)
}

len(osuked_to_account_id)

88

In [17]:
mapped_capacity_share = df_carbon_powerdict.loc[df_carbon_powerdict['osuked_id'].isin(osuked_to_account_id.keys()), 'capacity_mw'].sum()/df_carbon_powerdict['capacity_mw'].sum()

print(f'{mapped_capacity_share:.0%} of the CO2 emitting generation (with capacity data) in the power-station dictionary has been mapped to an EU ETS installation')

99% of the CO2 emitting generation (with capacity data) in the power-station dictionary has been mapped to an EU ETS installation


In [18]:
df_to_add_to_mapping = df_carbon_powerdict[(df_carbon_powerdict['osuked_id'].isin(s_osuked_ids_to_add))&(~df_carbon_powerdict['osuked_id'].isin(osuked_to_account_id.keys()))]

print(f'There are {df_to_add_to_mapping.shape[0]} plants left to add to the osuked_id to account_id mapping')

df_to_add_to_mapping

There are 2 plants left to add to the osuked_id to account_id mapping


,osuked_id,esail_id,gppd_idnr,name,sett_bmu_id,longitude,latitude,fuel_type,capacity_mw
80,10080,BRIDGWTR,NaN,Bridgewater District Energy,E_BRIDGWTR,-3.005964,51.127515,gas,NaN
126,10126,MANXENR,NaN,Manx Power,E_MANXENR-1,NaN,NaN,gas,NaN


In [35]:
sett_bmu_ids_with_eutl_mapping = flatten_list(pd.Series({v: k for k, v in osuked_to_account_id.items()}).map(df_powerdict.set_index('osuked_id')['sett_bmu_id'].to_dict()).str.split(', ').to_list())
ngc_bmu_ids_with_eutl_mapping = list(map(sett_to_ngc_bmu_id.get, sett_bmu_ids_with_eutl_mapping))

len(ngc_bmu_ids_with_eutl_mapping)

149

In [39]:
ngc_bmu_ids_without_eutl_mapping = list(set(df_B1610_sample.columns) - set(ngc_bmu_ids_with_eutl_mapping))

len(ngc_bmu_ids_without_eutl_mapping)

215

In [57]:
ngc_bmu_ids_with_no_fuel_mapping = set(ngc_bmu_ids_without_eutl_mapping) - set(ngc_bmu_id_to_fuel.keys())
s_ngc_bmu_fueltypes_unmatched_to_eutl = pd.Series(ngc_bmu_id_to_fuel).loc[set(ngc_bmu_ids_without_eutl_mapping)-set(ngc_bmu_ids_with_no_fuel_mapping)].value_counts()
s_ngc_bmu_fueltypes_unmatched_to_eutl.loc['unknown'] = len(ngc_bmu_ids_with_no_fuel_mapping)

s_ngc_bmu_fueltypes_unmatched_to_eutl

wind             108
hydro             37
nuclear           20
oil               10
coal               9
coal, biomass      6
gas                4
biomass            1
unknown           20
dtype: int64

In [ ]:
# create a list of the osuked_ids that are unmatched to eutl and are not wind, hydro, or nuclear

In [58]:
ngc_bmu_ids_with_no_fuel_mapping

{'ABRBO-1',
 'BEATO-1',
 'BEATO-2',
 'BEATO-3',
 'BEATO-4',
 'EAAO-1',
 'EAAO-2',
 'HOWAO-1',
 'HOWAO-2',
 'HOWAO-3',
 'KNLCV-1',
 'LCSMH-1',
 'MIDMW-1',
 'SEEL-1',
 'STLGW-1',
 'STLGW-2',
 'STLGW-3',
 'TLYMW-1',
 'WHIHW-1',
 'WILCT-1'}